# Sample and Visualize from Dataset and DataLoader instances

In [9]:
# from google.colab import drive
# drive.mount('/content/drive')

In [10]:
# !git clone https://github.com/ossnat/VSD_foundation_model.git
# !ln -s '/content/drive/My Drive/VSD_FM/Data' '/content'

In [11]:
import os
import sys
import yaml
import torch
import torch.nn.functional as F
from pathlib import Path
from torch.utils.data import DataLoader, Subset

# Resolve repo root when running from scripts/
project_root = Path.cwd().resolve().parent
sys.path.insert(0, str(project_root))

from src.data import load_dataset, create_dataset, VsdVideoDataset
from src.models.backbone.mae_backbone_2d import MAEResNet18Backbone
from src.models.heads.mae_decoder_2d import MAEDecoder2D
from src.models.systems.mae_system import MAESystem
from src.training.trainer import Trainer

# Load config
config_path = project_root / "configs" / "default.yaml"
with open(config_path, "r") as f:
    cfg = yaml.safe_load(f)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"\nUsing device: {device}")


Using device: cpu


## Visualize random sample from Dataset

In [12]:
# Initiate Dataset instacne
VSDdataset = VsdVideoDataset(cfg)

Loading split CSV from Data/FoundationData/ProcessedData/splits/split_v1_seed17_strat_monkey.csv...
Standard read failed: [Errno 2] No such file or directory: 'Data/FoundationData/ProcessedData/splits/split_v1_seed17_strat_monkey.csv', trying alternatives...


ValueError: Failed to read CSV file Data/FoundationData/ProcessedData/splits/split_v1_seed17_strat_monkey.csv. Last error: [Errno 2] No such file or directory: 'Data/FoundationData/ProcessedData/splits/split_v1_seed17_strat_monkey.csv'

In [ ]:
# Visualize a random sample before splitting
import random
import matplotlib.pyplot as plt
from src.utils.visualization import plot_frames_sequence, plot_spatial_dynamics

# Get a random sample
random_idx = random.randint(0, len(VSDdataset) - 1)
sample = VSDdataset[random_idx]
video_tensor = sample["video"]  # (C, T, H, W)
start_frame = sample.get("start_frame", 0)
end_frame = sample.get("end_frame", 0)

print(f"Visualizing random sample {random_idx} from dataset")
print(f"Video tensor shape: {video_tensor.shape}")
print(f"Frame range: {start_frame} to {end_frame}")

# Prepare real frame range for visualization
real_frame_range = (int(start_frame), int(end_frame)) if start_frame is not None and end_frame is not None else None

# Plot frames sequence
print("\n📊 Plotting frames sequence...")
fig1 = plot_frames_sequence(
    video_tensor=video_tensor,
    start_frame=0,
    end_frame=None,  # Use all frames
    clipping=(-0.003, 0.003),
    cols=10,
    title=f"Random Sample {random_idx} - Frame Sequence",
    show_plot=True,
    real_frame_range=real_frame_range
)

print("\n✅ Visualization complete!")


## Visualize from DataLoader

# Initiate DataLoader
batch_size=32
VSDloader = load_dataset(cfg, split="train", batch_size=batch_size, num_workers=0, shuffle=False)

In [ ]:
# Get a sample
sample = next(iter(VSDloader))
random_idx = random.randint(0, batch_size - 1)

video_tensor = sample["video"][random_idx,:,:,:,:].squeeze(0)  # Remove batch dimension to make it (C, T, H, W)
start_frame = sample.get("start_frame", 0)[random_idx]
end_frame = sample.get("end_frame", 0)[random_idx]

print(f"Visualizing random sample {random_idx} from dataset")
print(f"Video tensor shape: {video_tensor.shape}")
print(f"Frame range: {start_frame} to {end_frame}")

# Prepare real frame range for visualization
real_frame_range = (int(start_frame), int(end_frame)) if start_frame is not None and end_frame is not None else None

# Plot frames sequence
print("\n📊 Plotting frames sequence...")
fig1 = plot_frames_sequence(
    video_tensor=video_tensor,
    start_frame=0,
    end_frame=None,  # Use all frames
    clipping=(-0.003, 0.003),
    cols=10,
    title=f"Random Sample {random_idx} - Frame Sequence",
    show_plot=True,
    real_frame_range=real_frame_range
)

print("\n✅ Visualization complete!")